参考：
- <https://www.cnblogs.com/huchong/p/8244279.html>
- <https://segmentfault.com/a/1190000016497271>


单例模式
====

**单例模式（Singleton Pattern）**是一种常用的软件设计模式，该模式的主要目的是确保**某一个类只有一个实例存在**。当你希望在整个系统中，某个类只能出现一个实例时，单例对象就能派上用场。

比如，某个服务器程序的配置信息存放在一个文件中，客户端通过一个 AppConfig 的类来读取配置文件的信息。如果在程序运行期间，有很多地方都需要使用配置文件的内容，也就是说，很多地方都需要创建 AppConfig 对象的实例，这就导致系统中存在多个 AppConfig 的实例对象，而这样会严重浪费内存资源，尤其是在配置文件内容很多的情况下。事实上，类似 AppConfig 这样的类，我们希望在程序运行期间只存在一个实例对象。

在 Python 中，我们可以用多种方法来实现单例模式


实现单例模式的几种方式
===========

## 使用模块

其实，**Python 的模块就是天然的单例模式**，因为模块在第一次导入时，会生成 `.pyc` 文件，当第二次导入时，就会直接加载 `.pyc` 文件，而不会再次执行模块代码。因此，我们只需把相关的函数和数据定义在一个模块中，就可以获得一个单例对象了。如果我们真的想要一个单例类，可以考虑这样做：

**mysingleton.py**

```py
class Singleton(object):
    def foo(self):
        pass
singleton1 = Singleton()
```

将上面的代码保存在文件 `mysingleton.py` 中，要使用时，直接在其他任意文件中导入此文件中的对象，这个对象即是单例模式的对象，在其任一个模块中改变时，其他模块中的引用也会跟着改变。（引用类型）

```py
from mysingleton import singleton1
```


---

## 使用类

### 简洁方式

```py
class Singleton(object):

    def __init__(self):
        pass

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance
```
一般情况，大家以为这样就完成了单例模式，但是这样当使用多线程时会存在问题

In [23]:
class Singleton(object):

    def __init__(self):
        pass

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance

import threading

def task(arg):
    obj = Singleton.instance()
    print('%d  %s\n' %(arg, obj))

for i in range(10):
    t = threading.Thread(target=task, args=[i,])
    t.start()

0  <__main__.Singleton object at 0x000002351FEF94E0>

1  <__main__.Singleton object at 0x000002351FEF94E0>

2  <__main__.Singleton object at 0x000002351FEF94E0>

3  <__main__.Singleton object at 0x000002351FEF94E0>

4  <__main__.Singleton object at 0x000002351FEF94E0>

5  <__main__.Singleton object at 0x000002351FEF94E0>

6  <__main__.Singleton object at 0x000002351FEF94E0>

7  <__main__.Singleton object at 0x000002351FEF94E0>

8  <__main__.Singleton object at 0x000002351FEF94E0>

9  <__main__.Singleton object at 0x000002351FEF94E0>



<br>
看起来也没有问题，那是因为执行速度过快，如果在 init 方法中有一些 IO 操作，就会发现问题了，下面我们通过 time.sleep 模拟

我们在上面__init__方法中加入以下代码：

```py
def __init__(self):
    import time
    time.sleep(1)
```

重新执行程序后，结果如下

In [21]:
import time
import threading

class Singleton(object):

    def __init__(self):
        time.sleep(1)

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance


def task(arg):
    obj = Singleton.instance()
    print('%d  %s\n' %(arg, obj))

for i in range(10):
    t = threading.Thread(target=task,args=[i,])
    t.start()

time.sleep(15)

0  <__main__.Singleton object at 0x000002351FF71F98>

3  <__main__.Singleton object at 0x000002351FE40F28>

6  <__main__.Singleton object at 0x000002351FBD5F60>

2  <__main__.Singleton object at 0x000002351FE408D0>

7  <__main__.Singleton object at 0x000002351FEF9550>

8  <__main__.Singleton object at 0x000002351FEF9898>

1  <__main__.Singleton object at 0x000002351FE40F98>

9  <__main__.Singleton object at 0x000002351FEF94A8>

5  <__main__.Singleton object at 0x000002351FF65AC8>

4  <__main__.Singleton object at 0x000002351FF40358>



**问题出现了！按照以上方式创建的单例，无法支持多线程**

**解决办法：加锁！**未加锁部分并发执行, 加锁部分串行执行, 速度降低, 但是保证了数据安全

### 线程安全方式

In [4]:
import time
import threading

class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self):
        print('new')
        time.sleep(1)

    @classmethod
    def instance(cls, *args, **kwargs):
        with Singleton._instance_lock:
            if not hasattr(Singleton, "_instance"):
                Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance


def task(arg):
    obj = Singleton.instance()
    print('%d  %s\n' %(arg, obj))
    
for i in range(10):
    t = threading.Thread(target=task,args=[i,])
    t.start()
    
time.sleep(15)
obj = Singleton.instance()
print(obj)

new
0  <__main__.Singleton object at 0x00000287FFD4D550>
1  <__main__.Singleton object at 0x00000287FFD4D550>


2  <__main__.Singleton object at 0x00000287FFD4D550>
3  <__main__.Singleton object at 0x00000287FFD4D550>
4  <__main__.Singleton object at 0x00000287FFD4D550>



5  <__main__.Singleton object at 0x00000287FFD4D550>
6  <__main__.Singleton object at 0x00000287FFD4D550>
7  <__main__.Singleton object at 0x00000287FFD4D550>

8  <__main__.Singleton object at 0x00000287FFD4D550>



9  <__main__.Singleton object at 0x00000287FFD4D550>



### 💊进一步优化

这样就差不多了，但是还是有一点小问题，就是当程序执行时，执行了 time.sleep(20) 后，下面实例化对象时，此时已经是单例模式了，但我们还是加了锁，这样不太好，再进行一些优化，把 intance 方法，改成下面的这样就行：

```py
@classmethod
def instance(cls, *args, **kwargs):
    if not hasattr(Singleton, "_instance"):
        with Singleton._instance_lock:
            if not hasattr(Singleton, "_instance"):
                Singleton._instance = Singleton(*args, **kwargs)
    return Singleton._instance
```

这样，一个可以支持多线程的单例模式就完成了

In [25]:
import time
import threading

class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self):
        time.sleep(1)

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            with Singleton._instance_lock:
                if not hasattr(Singleton, "_instance"):
                    Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance


def task(arg):
    obj = Singleton.instance()
    print('%d  %s\n' %(arg, obj))
    
for i in range(10):
    t = threading.Thread(target=task,args=[i,])
    t.start()
    
time.sleep(15)
obj = Singleton.instance()
print(obj)


0  <__main__.Singleton object at 0x000002351FE631D0>
1  <__main__.Singleton object at 0x000002351FE631D0>

2  <__main__.Singleton object at 0x000002351FE631D0>

3  <__main__.Singleton object at 0x000002351FE631D0>

4  <__main__.Singleton object at 0x000002351FE631D0>

5  <__main__.Singleton object at 0x000002351FE631D0>

6  <__main__.Singleton object at 0x000002351FE631D0>

7  <__main__.Singleton object at 0x000002351FE631D0>

8  <__main__.Singleton object at 0x000002351FE631D0>

9  <__main__.Singleton object at 0x000002351FE631D0>




---
<br>

## 使用函数装饰器

In [1]:
def Singleton(cls):
    _instance = {}

    def _singleton(*args, **kargs):
        if cls not in _instance:
            _instance[cls] = cls(*args, **kargs)
        return _instance[cls]

    return _singleton


@Singleton
class A(object):

    def __init__(self, x=0):
        self.x = x


a1 = A(2)
a2 = A(3)
a1 is a2

True

代码中比较巧妙的一点是:

```py
_instance = {}
```

使用不可变的 **类地址** 作为键，其实例作为值，每次创造实例时，首先查看该类是否存在实例，存在的话直接返回该实例即可，否则新建一个实例并存放在字典中。

**但是在未加锁的情况下也是不安全的。**


## 使用类装饰器

In [6]:
class Singleton(object):
    _instance = {}
    
    def __init__(self, cls):
        self._cls = cls

    def __call__(self):
        if self._cls not in self._instance:
            self._instance[self._cls] = self._cls()
        return self._instance[self._cls]


@Singleton
class Cls2(object):
    def __init__(self):
        pass


cls1 = Cls2()
cls2 = Cls2()
cls3 = Cls2()
print(id(cls1) == id(cls2))
print(cls1 is cls3)

True
True


同时，由于是面对对象的，这里还可以这么用

In [3]:
class Cls3():
    pass

Cls3 = Singleton(Cls3)
cls3 = Cls3()
cls4 = Cls3()
print(id(cls3) == id(cls4))

True


使用 类装饰器实现单例的原理和 函数装饰器 实现的原理相似，理解了上文，再理解这里应该不难。

<br>

**注意：这种方式实现的单例模式，使用时会有限制，以后实例化必须通过 `obj = Singleton.instance()`，如果用 `obj=Singleton()` , 这种方式得到的不是单例**

**在未加锁的情况下也是不安全的。**

---

---

## New、Metaclass 关键字

在接着说另外两种方法之前，需要了解在 Python 中一个类和一个实例是通过哪些方法以怎样的顺序被创造的。

简单来说， 元类 ( metaclass ) 可以通过方法 metaclass 创造了 类(class) ，而 类(class) 通过方法 new 创造了 实例(instance) 。

在单例模式应用中，在创造类的过程中或者创造实例的过程中稍加控制达到最后产生的实例都是一个对象的目的。

本文主讲单例模式，所以对这个 topic 只会点到为止，有感兴趣的同学可以在网上搜索相关内容，几篇参考文章：

- [What are metaclasses in Python?](https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python)
- [python-\__new__-magic-method-explained](http://howto.lintel.in/python-__new__-magic-method-explained/)
- [Why is __init__() always called after __new__()?](https://stackoverflow.com/questions/674304/why-is-init-always-called-after-new)

---

## 基于`__new__`方法实现

（推荐使用，方便）

通过上面例子，我们可以知道，当我们实现单例时，**为了保证线程安全需要在内部加入锁**

我们知道，当我们实例化一个对象时，是**先执行了类的`__new__`方法**（我们没写时，默认调用 `object.__new__`），**实例化对象**；然后**再执行类的`__init__`方法**，对这个对象进行初始化，所以我们可以基于这个，实现单例模式,

**但是 `__init__()` 方法还是每次都会执行**

In [1]:
import threading

# Method 1:
# class Single(object):
#     _instance = None
#     def __new__(cls, *args, **kw):
#         if cls._instance is None:
#             cls._instance = object.__new__(cls, *args, **kw)
#         return cls._instance
#     def __init__(self):
#         pass

# Method 2:
class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self):
        print('new')
        pass

    def __new__(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            with Singleton._instance_lock:
                if not hasattr(Singleton, "_instance"):
                    Singleton._instance = object.__new__(cls)
        return Singleton._instance


obj1 = Singleton()
obj2 = Singleton()
print(obj1 is obj2)

new
new
True


In [27]:
def task(arg):
    obj = Singleton()
    print('%d  %s\n' %(arg, obj))


for i in range(10):
    t = threading.Thread(target=task, args=[i, ])
    t.start()

0  <__main__.Singleton object at 0x000002352F813F60>

1  <__main__.Singleton object at 0x000002352F813F60>

2  <__main__.Singleton object at 0x000002352F813F60>
3  <__main__.Singleton object at 0x000002352F813F60>


4  <__main__.Singleton object at 0x000002352F813F60>

5  <__main__.Singleton object at 0x000002352F813F60>

6  <__main__.Singleton object at 0x000002352F813F60>

7  <__main__.Singleton object at 0x000002352F813F60>

8  <__main__.Singleton object at 0x000002352F813F60>

9  <__main__.Singleton object at 0x000002352F813F60>



采用这种方式的单例模式，以后实例化对象时，和平时实例化对象的方法一样 `obj = Singleton()`

---

## 基于 metaclass 方式实现

### 相关知识

1. 类由type创建，创建类时，type的`__init__`方法自动执行，`类()` 执行type的 `__call__`方法(类的`__new__`方法,类的`__init__`方法)
2. 对象由类创建，创建对象时，类的`__init__`方法自动执行，`对象()` 执行类的 `__call__` 方法

In [30]:
class Foo:
    def __init__(self):
        pass

    def __call__(self, *args, **kwargs):
        pass

    
# 执行type的 __call__ 方法，调用 Foo类（是type的对象）的 __new__方法，
# 用于创建对象，然后调用 Foo类（是type的对象）的 __init__方法，用于对对象初始化。
obj = Foo()


obj()    # 执行Foo的 __call__ 方法

### 元类的使用

In [4]:
class SingletonType(type):
    
    def __init__(self, *args, **kwargs):
        super(SingletonType, self).__init__(*args, **kwargs)

    def __call__(cls, *args, **kwargs):  # 这里的cls，即Foo类
        print('cls', cls)
        obj = cls.__new__(cls, *args, **kwargs)
        cls.__init__(obj, *args, **kwargs)  # Foo.__init__(obj)
        return obj


class Foo(metaclass=SingletonType):  # 指定创建Foo的type为SingletonType
    def __init__(self, name):
        self.name = name

    def __new__(cls, *args, **kwargs):
        return object.__new__(cls)


obj1 = Foo('Foo1')

cls <class '__main__.Foo'>


<br>

### 使用 type 创造类

在实现单例之前，需要了解**使用 `type` 创造类**的方法，代码如下：

In [12]:
def func(self):
    print("do sth")

Klass = type("Klass", (), {"func": func})

c = Klass()
c.func()

do sth


以上，我们使用 type 创造了一个类出来。这里的知识是 mataclass 实现单例的基础。

### 实现单例模式

In [5]:
# Method 1
import threading


class SingletonType(type):
    _instance_lock = threading.Lock()

    def __call__(cls, *args, **kwargs):
        if not hasattr(cls, "_instance"):
            with SingletonType._instance_lock:
                if not hasattr(cls, "_instance"):
                    cls._instance = super(SingletonType,
                                          cls).__call__(*args, **kwargs)
        return cls._instance


class Foo(metaclass=SingletonType):
    def __init__(self, name):
        self.name = name


obj1 = Foo('Foo1')
obj2 = Foo('Foo2')
print(obj1 is obj2)
print(obj2.name)


# 可以应用到多个单例中
class Faa(metaclass=SingletonType):
    def __init__(self, name):
        self.name = name


obj1 = Faa('Faa1')
obj2 = Faa('Faa2')
print(obj1 is obj2)
print(obj2.name)

True
Foo1
True
Faa1


In [17]:
# Method 2

class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton,
                                        cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Cls4(metaclass=Singleton):
    pass


cls1 = Cls4()
cls2 = Cls4()
print(id(cls1) == id(cls2))

True
